## Demo

In [2]:
from main import LateEmbedding

text = [
    "谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团",
    "解放战争时期，任陕甘宁联防军政治部组织部副部长，联防军后勤部政治部主任，冀鲁豫军区政治部组织部部长",
    "他因战伤截肢，人称“独脚将军。”获一级八一勋章、二级独立自由勋章、一级解放勋章和一级红星功勋荣誉章。"
]

embeddings = LateEmbedding().main(text)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
for i, (e, t) in enumerate(zip(embeddings, text)):
    print("文本：", t)
    print("Embedding：", e)

文本： 谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团
embedding： [-1.1230367   1.0277416  -0.6005747   0.6233702   0.05251494 -1.0940877
  0.1847056   0.39331266  1.2308713  -0.8029521  -0.53582084  0.2045242
 -0.6924701   0.5315508  -0.8010977   0.6209492   1.0815803   1.016102
  1.0749965   0.7177181   0.9415335   0.73199433 -0.26983666  0.21583831
  0.08235107  0.46477     0.38571724 -0.991843   -0.40135434  0.76391953
 -0.53406596 -0.10234588  0.18440096  0.35458454  1.2977713  -0.29474872
  0.5412565   0.5016636  -0.31964955  0.016714   -0.5683658  -0.67063475
  0.6545851   0.79328173 -0.54971105  0.14478526 -0.75414485  0.28846532
 -0.6284936   0.98550826  0.137964    0.34663963  0.48332134  0.65366155
  0.6720803   0.5093222  -1.719553    0.36982727  1.0655748   0.06760635
  0.25775614 -0.19801764 -0.03310338 -0.5272405  -0.00422603 -0.18433525
  0.4120313   0.4203751   2.040147    0.09006051 -0.3928837   0.9659426
 -0.9455704   0.5635552   0.65643674  0.08160163 -0.7796615

## 比较传统chunking和late chunking效果

In [17]:
text = [
    "谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团",
    "解放战争时期，任陕甘宁联防军政治部组织部副部长，联防军后勤部政治部主任，冀鲁豫军区政治部组织部部长",
    "他因战伤截肢，人称“独脚将军。”获一级八一勋章、二级独立自由勋章、一级解放勋章和一级红星功勋荣誉章。"
]

In [ ]:
# 传统chunking方法
from sentence_transformers import SentenceTransformer

trditional_model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)
embeddings_traditional_chunking = trditional_model.encode(text)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# late chunking方法
embeddings = LateEmbedding().main(text)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
# 评估cosine similarity
import numpy as np

cos_sim = lambda x, y: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
query = '谁被称为称独脚将军'
query_embedding = trditional_model.encode(query)

print(f'query: {query}')
print("===")
for chunk, late_embedding, trad_embedding in zip(text, embeddings, embeddings_traditional_chunking):
    print(cos_sim(query_embedding, trad_embedding), f'trational: ("{chunk}"):')
    print(cos_sim(query_embedding, late_embedding), f'late: ("{chunk}"):')
    print("===")

query: 谁被称为称独脚将军
===
0.5242377 trational: ("谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团"):
0.5062333 late: ("谢良兴国县长冈乡塘石村人。生于一九一五年四月。一九三○年六月参加中国工农红军，同年七月加人中国共产主义青年团"):
===
0.53565854 trational: ("解放战争时期，任陕甘宁联防军政治部组织部副部长，联防军后勤部政治部主任，冀鲁豫军区政治部组织部部长"):
0.44494602 late: ("解放战争时期，任陕甘宁联防军政治部组织部副部长，联防军后勤部政治部主任，冀鲁豫军区政治部组织部部长"):
===
0.85351825 trational: ("他因战伤截肢，人称“独脚将军。”获一级八一勋章、二级独立自由勋章、一级解放勋章和一级红星功勋荣誉章。"):
0.46660474 late: ("他因战伤截肢，人称“独脚将军。”获一级八一勋章、二级独立自由勋章、一级解放勋章和一级红星功勋荣誉章。"):
===
